In [1]:
using LinearAlgebra

In [2]:
function house(x)
    sigma = dot(x[2:end], x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq      # reflect to positive x axis
    else
        v[1] -= sq      # reflect to negative x axis
    end

    beta = 2.0 * v[1]^2 / (v[1]^2 + sigma)
    v1 = v[1]

    for i=1:length(v)
        v[i] /= v1
    end
    # display(v)
        
    return beta, transpose(v)   # get column vector v
end

house (generic function with 1 method)

In [3]:
x = [-10.0 5.0]

beta, v = house(x)
;

In [4]:
reflected_x = (Matrix(I, 2, 2) - beta * (v * transpose(v))) * x'

@show v'
@show reflected_x'
;

v' = [1.0 -0.2360679774997897]


reflected_x' = [11.180339887498949 -3.3306690738754696e-16]


In [5]:
function givens(a, b)
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a / b
            s = 1.0 / sqrt(1.0 + tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0 / sqrt(1.0 + tau*tau)
            s = c*tau
        end
    end
    return (c, s)
end

givens (generic function with 1 method)

# Using householder

In [6]:
using LinearAlgebra
using Random
rng = MersenneTwister(5)

Random.seed!(rng, 69)

m, n = 10, 5
A = Float64.(rand(rng, -10:10, m, n))

10×5 Matrix{Float64}:
   9.0    3.0   3.0  -8.0   -1.0
   0.0  -10.0  -8.0  -3.0    4.0
   9.0    1.0  -2.0   4.0    4.0
   2.0   -9.0  -9.0   8.0   -2.0
 -10.0   -3.0  -6.0  -2.0  -10.0
  -5.0   -5.0  10.0  -2.0  -10.0
  -7.0   -6.0   1.0   4.0    3.0
   4.0    1.0  -9.0  -9.0   -8.0
  -4.0    5.0  10.0  -9.0   -4.0
  -2.0    0.0   6.0  -8.0   -3.0

In [7]:
function factorize_QR_house(A)
    m, n = size(A)
    A_out = copy(A)

    for j=1:n
        vTA = zeros(n)      # stores beta * vT * A
        beta, v = house(A[j:end, j]) 
        # v is of shape (m-j+1 x 1), same as the lower jth column of A

        for k=j:n       # row indices of A, starts from diagonal element
            for i=j:m   
                vTA[k] += v[i-j+1] * A[i,k]
            end
            vTA[k] *= beta
        end

        for k=j:n
            for i=j:m
                A[i,k] -= v[i-j+1] * vTA[k]
            end
        end
        
        display(j)
        display(round.(A, digits=10))
        display(v)
        A_out[j+1:end, j] = v[2:end]
    end

    return A_out
end

factorize_QR_house (generic function with 1 method)

In [8]:
Aprime = factorize_QR_house(A)
;

1

10×5 Matrix{Float64}:
 -19.3907   -5.10554    4.84768     0.103142  -7.32309
   0.0     -10.0       -8.0        -3.0        4.0
   0.0      -1.5695    -1.41428     6.56874    1.99555
   0.0      -9.571     -8.86984     8.57083   -2.44543
  -0.0      -0.145005  -6.6508     -4.85415   -7.77283
  -0.0      -3.5725     9.6746     -3.42708   -8.88642
  -0.0      -4.0015     0.544437    2.00209    4.55902
   0.0      -0.141998  -8.73968    -7.85834   -8.89087
   0.0       6.142      9.73968   -10.1417    -3.10913
   0.0       0.570999   5.86984    -8.57083   -2.55457

1×10 transpose(::Vector{Float64}) with eltype Float64:
 1.0  0.0  0.317005  0.0704456  -0.352228  …  0.140891  -0.140891  -0.0704456

2

10×5 Matrix{Float64}:
 -19.3907  -5.10554   4.84768    0.103142  -7.32309
   0.0     16.1534   12.1182    -7.64379   -1.50979
   0.0      0.0      -0.206964   6.29006    1.6649
   0.0      0.0      -1.50748    6.87141   -4.46177
  -0.0      0.0      -6.53926   -4.8799    -7.80338
  -0.0      0.0      12.4227    -4.06141   -9.63904
  -0.0      0.0       3.62254    1.29159    3.71601
   0.0      0.0      -8.63045   -7.88355   -8.92078
   0.0      0.0       5.01503   -9.05109   -1.81519
   0.0      0.0       5.43061   -8.46944   -2.43427

...

10×5 Matrix{Float64}:
 -19.3907  -5.10554   4.84768    0.103142  -7.32309
   0.0     16.1534   12.1182    -7.64379   -1.50979
   0.0      0.0      18.4838    -2.64401    0.313566
   0.0      0.0      -0.0      -18.3731    -9.25504
  -0.0      0.0      -0.0        0.0       -6.74702
  -0.0      0.0       0.0       -0.0       -9.09933
  -0.0      0.0       0.0        0.0        3.40048
   0.0      0.0      -0.0        0.0       -7.25097
   0.0      0.0       0.0        0.0       -0.181645
   0.0      0.0       0.0        0.0       -0.919727

1×7 transpose(::Vector{Float64}) with eltype Float64:
 1.0  -0.326441  0.07652  0.123273  -0.489679  -0.271324  -0.239506

5

10×5 Matrix{Float64}:
 -19.3907  -5.10554   4.84768    0.103142  -7.32309
   0.0     16.1534   12.1182    -7.64379   -1.50979
   0.0      0.0      18.4838    -2.64401    0.313566
   0.0      0.0      -0.0      -18.3731    -9.25504
  -0.0      0.0      -0.0        0.0       13.9046
  -0.0      0.0       0.0       -0.0        0.0
  -0.0      0.0       0.0        0.0        0.0
   0.0      0.0      -0.0        0.0       -0.0
   0.0      0.0       0.0        0.0        0.0
   0.0      0.0       0.0        0.0        0.0

1×6 transpose(::Vector{Float64}) with eltype Float64:
 1.0  0.44061  -0.164659  0.351108  0.00879568  0.0445353

# Using givens

In [9]:
using LinearAlgebra
using Random
rng = MersenneTwister(5)

Random.seed!(rng, 68)

m, n = 10, 5
A = triu(Float64.(rand(rng, -10:10, m, n)), -1)

10×5 Matrix{Float64}:
 -5.0  -3.0  8.0   -9.0   1.0
 -6.0  -7.0  6.0   -9.0  -8.0
  0.0   8.0  2.0  -10.0   0.0
  0.0   0.0  1.0  -10.0   1.0
  0.0   0.0  0.0    4.0  10.0
  0.0   0.0  0.0    0.0  -3.0
  0.0   0.0  0.0    0.0   0.0
  0.0   0.0  0.0    0.0   0.0
  0.0   0.0  0.0    0.0   0.0
  0.0   0.0  0.0    0.0   0.0

In [10]:
function factorize_QR_givens(A)
    for j=1:n
        c, s = givens(A[j,j], A[j+1,j])

        for k=j:n
            A[j,k], A[j+1,k] = c * A[j,k] - s * A[j+1,k], s * A[j,k] + c * A[j+1,k]
        end
        display(j)
        display(round.(A, digits=10))
    end
end

factorize_QR_givens (generic function with 1 method)

In [11]:
factorize_QR_givens(A)

1

10×5 Matrix{Float64}:
 7.81025  7.2981   -9.7308    12.6757    5.50559
 0.0      2.17663   2.30466   -1.15233   5.8897
 0.0      8.0       2.0      -10.0       0.0
 0.0      0.0       1.0      -10.0       1.0
 0.0      0.0       0.0        4.0      10.0
 0.0      0.0       0.0        0.0      -3.0
 0.0      0.0       0.0        0.0       0.0
 0.0      0.0       0.0        0.0       0.0
 0.0      0.0       0.0        0.0       0.0
 0.0      0.0       0.0        0.0       0.0

2

10×5 Matrix{Float64}:
 7.81025   7.2981   -9.7308    12.6757    5.50559
 0.0      -8.29082  -2.5349     9.95175  -1.54625
 0.0       0.0       1.69875    1.51343   5.6831
 0.0       0.0       1.0      -10.0       1.0
 0.0       0.0       0.0        4.0      10.0
 0.0       0.0       0.0        0.0      -3.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0

3

10×5 Matrix{Float64}:
 7.81025   7.2981   -9.7308   12.6757    5.50559
 0.0      -8.29082  -2.5349    9.95175  -1.54625
 0.0       0.0       1.97123  -3.76873   5.40483
 0.0       0.0      -0.0      -9.38548  -2.02125
 0.0       0.0       0.0       4.0      10.0
 0.0       0.0       0.0       0.0      -3.0
 0.0       0.0       0.0       0.0       0.0
 0.0       0.0       0.0       0.0       0.0
 0.0       0.0       0.0       0.0       0.0
 0.0       0.0       0.0       0.0       0.0

4

10×5 Matrix{Float64}:
 7.81025   7.2981   -9.7308    12.6757    5.50559
 0.0      -8.29082  -2.5349     9.95175  -1.54625
 0.0       0.0       1.97123   -3.76873   5.40483
 0.0       0.0      -0.0      -10.2023   -5.7801
 0.0       0.0       0.0        0.0       8.4069
 0.0       0.0       0.0        0.0      -3.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0

5

10×5 Matrix{Float64}:
 7.81025   7.2981   -9.7308    12.6757    5.50559
 0.0      -8.29082  -2.5349     9.95175  -1.54625
 0.0       0.0       1.97123   -3.76873   5.40483
 0.0       0.0      -0.0      -10.2023   -5.7801
 0.0       0.0       0.0        0.0       8.92614
 0.0       0.0       0.0        0.0      -0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0
 0.0       0.0       0.0        0.0       0.0